**Tutorial NER dengan Metode CRF**

Sumber: 

https://towardsdatascience.com/named-entity-recognition-and-classification-with-scikit-learn-f05372f07ba2

https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html



Install library CRF

In [7]:
!pip install sklearn_crfsuite

You should consider upgrading via the 'd:\python3.7\python.exe -m pip install --upgrade pip' command.


Import library yang diperlukan


In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.metrics import classification_report

Baca dataset. Link file dataset: https://drive.google.com/file/d/17Hm26nL6EQ8aDrr_Abz3KqUkhnxa_Uoi/view?usp=sharing 

Jangan lupa sesuaikan path!

In [9]:
df = pd.read_csv('data/ner_100000_dataset.csv', encoding = "ISO-8859-1")

df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/ner_100000_dataset.csv'

Preprocess, update NaN pada kolom Sentence # dengan nomor kalimat yang ada di baris terakhir sebelumnya.

In [ ]:
df = df.fillna(method='ffill')
df['Sentence #'].nunique(), df.Word.nunique(), df.Tag.nunique()

Cek persebaran tag. Dapat dilihat jika persebarannya tidak seimbang (imbalanced)

In [ ]:
df.groupby('Tag').size().reset_index(name='counts')

Class untuk mengambil data per sentence

In [ ]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(), 
                                                           s['POS'].values.tolist(), 
                                                           s['Tag'].values.tolist())]
        self.grouped = self.data.groupby('Sentence #').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None
getter = SentenceGetter(df)
sentences = getter.sentences

Fungsi untuk ekstraksi fitur

In [ ]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:], #Komen ini, Tanpa Akhiran
        'word[-2:]': word[-2:], #Komen ini, Tanpa Akhiran
        'word.isupper()': word.isupper(), #Komen ini, Tanpa Ortografi
        'word.istitle()': word.istitle(), #Komen ini, Tanpa Ortografi
        'word.isdigit()': word.isdigit(), #Komen ini, Tanpa Ortografi
        'postag': postag, #Komen ini, Tanpa Postag
        'postag[:2]': postag[:2], #Komen ini, Tanpa Postag
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(), #Komen ini, Tanpa Ortografi
            '-1:word.isupper()': word1.isupper(), #Komen ini, Tanpa Ortografi
            '-1:postag': postag1, #Komen ini, Tanpa Postag
            '-1:postag[:2]': postag1[:2], #Komen ini, Tanpa Postag
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(), #Komen ini, Tanpa Ortografi
            '+1:word.isupper()': word1.isupper(), #Komen ini, Tanpa Ortografi
            '+1:postag': postag1, #Komen ini, Tanpa Postag
            '+1:postag[:2]': postag1[:2], #Komen ini, Tanpa Postag
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]
    
def sent2tokens(sent):
    return [token for token, postag, label in sent]

Kelas/label tag, diproses supaya nanti label O tidak diperhitungkan untuk evaluasi

In [ ]:
y = df.Tag.values
classes = np.unique(y)
classes = classes.tolist()
new_classes = classes.copy()
new_classes.pop()
new_classes

Ekstraksi fitur dan split data menjadi data train dan data test

In [ ]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

Cek salah satu instance di data train

In [ ]:
# print(X_train[0])
# print(y_train[0])

Pelatihan dengan metode CRF

In [ ]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

Prediksi label data test.

Print metriks evaluasi.

In [ ]:
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(y_test, y_pred, labels = new_classes))